In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('/content/Churn_Modelling.csv')

In [3]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
df.dtypes

RowNumber            int64
CustomerId           int64
Surname             object
CreditScore          int64
Geography           object
Gender              object
Age                  int64
Tenure               int64
Balance            float64
NumOfProducts        int64
HasCrCard            int64
IsActiveMember       int64
EstimatedSalary    float64
Exited               int64
dtype: object

In [6]:
df.drop(['RowNumber','CustomerId','Surname'],axis=1,inplace=True)

In [7]:
df.columns

Index(['CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance',
       'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary',
       'Exited'],
      dtype='object')

In [8]:
df.shape

(10000, 11)

In [9]:
df.isnull().sum()

CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [10]:
def unique_values(df):
  for col in df:
    if(df[col].dtypes == 'object'):
      print(f'{col}: {df[col].unique()}')

In [11]:
unique_values(df)

Geography: ['France' 'Spain' 'Germany']
Gender: ['Female' 'Male']


In [13]:
df['Gender'].replace({'Female':1,'Male':0},inplace=True)

In [14]:
df1 = pd.get_dummies(data=df, columns=['Geography'])
df1.columns

Index(['CreditScore', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts',
       'HasCrCard', 'IsActiveMember', 'EstimatedSalary', 'Exited',
       'Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype='object')

In [15]:
df1.sample(5)

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
4338,714,0,25,2,0.00,1,1,1,132979.43,0,0,0,1
4527,705,0,38,1,189443.72,1,0,1,106648.58,0,1,0,0
8976,720,0,29,2,0.00,2,1,0,39925.52,0,1,0,0
5295,706,1,29,6,185544.36,1,1,0,171037.63,0,0,1,0
4753,454,0,40,2,123177.01,1,1,0,148309.98,0,0,0,1


In [16]:
cols_to_scale = ['Tenure','CreditScore','Balance','EstimatedSalary']

from sklearn.preprocessing import MinMaxScaler 
scaler = MinMaxScaler()

df1[cols_to_scale] = scaler.fit_transform(df1[cols_to_scale])

In [17]:
df1.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,0.538,1,42,0.2,0.000000,1,1,1,0.506735,1,1,0,0
1,0.516,1,41,0.1,0.334031,1,0,1,0.562709,0,0,0,1
2,0.304,1,42,0.8,0.636357,3,1,0,0.569654,1,1,0,0
3,0.698,1,39,0.1,0.000000,2,0,0,0.469120,0,1,0,0
4,1.000,1,43,0.2,0.500246,1,1,1,0.395400,0,0,0,1


In [19]:
X = df1.drop(['Exited'],axis=1)
y = df1.Exited

In [20]:
X.shape

(10000, 12)

In [21]:
y.shape

(10000,)

In [25]:
X.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,0.538,1,42,0.2,0.000000,1,1,1,0.506735,1,0,0
1,0.516,1,41,0.1,0.334031,1,0,1,0.562709,0,0,1
2,0.304,1,42,0.8,0.636357,3,1,0,0.569654,1,0,0
3,0.698,1,39,0.1,0.000000,2,0,0,0.469120,1,0,0
4,1.000,1,43,0.2,0.500246,1,1,1,0.395400,0,0,1


In [22]:
y.value_counts()

0    7963
1    2037
Name: Exited, dtype: int64

In [23]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=15, stratify=y)

In [24]:
from sklearn.metrics import confusion_matrix , classification_report

In [26]:
def ANN(X_train, y_train, X_test, y_test, loss, weights):
    model = keras.Sequential([
        keras.layers.Dense(20, input_dim=12, activation='relu'),
        keras.layers.Dense(15, activation='relu'),
        keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])
    
    if weights == -1:
        model.fit(X_train, y_train, epochs=100)
    else:
        model.fit(X_train, y_train, epochs=100, class_weight = weights)
    
    print(model.evaluate(X_test, y_test))
    
    y_preds = model.predict(X_test)
    y_preds = np.round(y_preds)
    
    print("Classification Report: \n", classification_report(y_test, y_preds))
    
    return y_preds

In [35]:
y_preds = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy', -1)

Epoch 1/100
250/250 [==============================] - 1s 2ms/step - loss: 0.5297 - accuracy: 0.7962
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4823 - accuracy: 0.7970
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4587 - accuracy: 0.8030
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4496 - accuracy: 0.8061
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4429 - accuracy: 0.8102
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4402 - accuracy: 0.8106
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4369 - accuracy: 0.8094
Epoch 8/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4372 - accuracy: 0.8120
Epoch 9/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4380 - accuracy: 0.8125
Epoch 10/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4350 - accura

**UnderSampling**

In [28]:
# Class count
count_class_0, count_class_1 = df1.Exited.value_counts()

# Divide by class
df_class_0 = df1[df1['Exited'] == 0]
df_class_1 = df1[df1['Exited'] == 1]

In [29]:
count_class_0

7963

In [30]:
count_class_1

2037

In [31]:
df_class_0_under = df_class_0.sample(count_class_1)

df_test_under = pd.concat([df_class_0_under, df_class_1], axis=0)
df_test_under.shape

(4074, 13)

In [32]:
df_test_under.Exited.value_counts()

0    2037
1    2037
Name: Exited, dtype: int64

In [33]:
X_under = df_test_under.drop('Exited',axis='columns')
y_under = df_test_under['Exited']

from sklearn.model_selection import train_test_split
X_train_under, X_test_under, y_train_under, y_test_under = train_test_split(X_under, y_under, test_size=0.2, random_state=15, stratify=y_under)

In [34]:
y_preds = ANN(X_train_under, y_train_under, X_test_under, y_test_under, 'binary_crossentropy', -1)

Epoch 1/100
102/102 [==============================] - 1s 2ms/step - loss: 0.6739 - accuracy: 0.5790
Epoch 2/100
102/102 [==============================] - 0s 2ms/step - loss: 0.6409 - accuracy: 0.6462
Epoch 3/100
102/102 [==============================] - 0s 2ms/step - loss: 0.6208 - accuracy: 0.6637
Epoch 4/100
102/102 [==============================] - 0s 2ms/step - loss: 0.6078 - accuracy: 0.6723
Epoch 5/100
102/102 [==============================] - 0s 2ms/step - loss: 0.6011 - accuracy: 0.6769
Epoch 6/100
102/102 [==============================] - 0s 2ms/step - loss: 0.5967 - accuracy: 0.6818
Epoch 7/100
102/102 [==============================] - 0s 2ms/step - loss: 0.5913 - accuracy: 0.6864
Epoch 8/100
102/102 [==============================] - 0s 2ms/step - loss: 0.5932 - accuracy: 0.6867
Epoch 9/100
102/102 [==============================] - 0s 2ms/step - loss: 0.5882 - accuracy: 0.6876
Epoch 10/100
102/102 [==============================] - 0s 2ms/step - loss: 0.5885 - accura

In [37]:
#for class 1
"""
precision and recall improved:
before: 0.85      0.36      0.51       407
now:    0.75      0.68      0.71       407

"""

'\nprecision and recall improved:\nbefore: 0.85      0.36      0.51       407\nnow:    0.75      0.68      0.71       407\n\n'

**OverSampling**

In [43]:
count_class_0

7963

In [44]:
count_class_1

2037

In [41]:
df_class_0.shape

(7963, 13)

In [42]:
df_class_1.shape

(2037, 13)

In [48]:
#df_class_0 () , df_class_1 exist

df_class_1_over = df_class_1.sample(count_class_0,replace=True)

df_test_over = pd.concat([df_class_0, df_class_1_over], axis=0)
df_test_over.shape

(15926, 13)

In [49]:
df_test_over.Exited.value_counts()

0    7963
1    7963
Name: Exited, dtype: int64

In [50]:
X_over = df_test_over.drop(['Exited'],axis=1)
y_over = df_test_over.Exited

In [51]:
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over, test_size=0.2, random_state=15, stratify=y_over)

In [52]:
y_preds = ANN(X_train_over, y_train_over, X_test_over, y_test_over, 'binary_crossentropy', -1)

Epoch 1/100
399/399 [==============================] - 2s 3ms/step - loss: 0.6883 - accuracy: 0.5763
Epoch 2/100
399/399 [==============================] - 1s 3ms/step - loss: 0.6080 - accuracy: 0.6814
Epoch 3/100
399/399 [==============================] - 1s 2ms/step - loss: 0.5906 - accuracy: 0.6953
Epoch 4/100
399/399 [==============================] - 1s 2ms/step - loss: 0.5879 - accuracy: 0.6957
Epoch 5/100
399/399 [==============================] - 1s 2ms/step - loss: 0.5840 - accuracy: 0.6952
Epoch 6/100
399/399 [==============================] - 1s 2ms/step - loss: 0.5837 - accuracy: 0.6953
Epoch 7/100
399/399 [==============================] - 1s 2ms/step - loss: 0.5804 - accuracy: 0.7005
Epoch 8/100
399/399 [==============================] - 1s 2ms/step - loss: 0.5803 - accuracy: 0.6965
Epoch 9/100
399/399 [==============================] - 1s 2ms/step - loss: 0.5795 - accuracy: 0.6999
Epoch 10/100
399/399 [==============================] - 1s 2ms/step - loss: 0.5786 - accura

In [53]:
"""
For class 1 

precision and recall improved:
before:            0.85      0.36      0.51       407
under_sampling:    0.75      0.68      0.71       407
over_sampling:     0.77      0.77      0.77      1593

"""

'\nFor class 1 \n\nprecision and recall improved:\nbefore:            0.85      0.36      0.51       407\nunder_sampling:    0.75      0.68      0.71       407\nover_sampling:     0.77      0.77      0.77      1593\n\n'

**SMOTE**

In [54]:
X = df1.drop('Exited',axis=1)
y = df1.Exited

In [56]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(sampling_strategy='minority')
X_sm, y_sm = smote.fit_resample(X, y)

y_sm.value_counts()

1    7963
0    7963
Name: Exited, dtype: int64

In [57]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_sm, y_sm, test_size=0.2, random_state=15, stratify=y_sm)

In [58]:
y_preds = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy', -1)

Epoch 1/100
399/399 [==============================] - 2s 3ms/step - loss: 0.6567 - accuracy: 0.6396
Epoch 2/100
399/399 [==============================] - 1s 3ms/step - loss: 0.5705 - accuracy: 0.7069
Epoch 3/100
399/399 [==============================] - 1s 3ms/step - loss: 0.5644 - accuracy: 0.7061
Epoch 4/100
399/399 [==============================] - 1s 2ms/step - loss: 0.5665 - accuracy: 0.7054
Epoch 5/100
399/399 [==============================] - 1s 2ms/step - loss: 0.5625 - accuracy: 0.7074
Epoch 6/100
399/399 [==============================] - 1s 2ms/step - loss: 0.5626 - accuracy: 0.7064
Epoch 7/100
399/399 [==============================] - 1s 2ms/step - loss: 0.5611 - accuracy: 0.7106
Epoch 8/100
399/399 [==============================] - 1s 2ms/step - loss: 0.5602 - accuracy: 0.7091
Epoch 9/100
399/399 [==============================] - 1s 2ms/step - loss: 0.5634 - accuracy: 0.7087
Epoch 10/100
399/399 [==============================] - 1s 2ms/step - loss: 0.5591 - accura

In [59]:
"""
For class 1 

precision and recall improved:
before:            0.85      0.36      0.51       407
under_sampling:    0.75      0.68      0.71       407
over_sampling:     0.77      0.77      0.77      1593
SMOTE(os):         0.76      0.84      0.80      1593

"""

'\nFor class 1 \n\nprecision and recall improved:\nbefore:            0.85      0.36      0.51       407\nunder_sampling:    0.75      0.68      0.71       407\nover_sampling:     0.77      0.77      0.77      1593\nSMOTE(os):         0.76      0.84      0.80      1593\n\n'